### Lab2 (a) Model preperation

In [1]:
from resnet20 import ResNetCIFAR
from train_util import train, finetune, test
import torch
import numpy as np

import time

import torchvision.transforms as transforms
import torchvision
import torch.nn as nn
import torch.optim as optim

from FP_layers import *

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)

In [3]:
# Load the best weight paramters
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

100%|██████████| 170M/170M [00:02<00:00, 58.0MB/s]


Test Loss=0.3231, Test accuracy=0.9151


The test accuracy for the floating point pretrained model is 0.9151.

### Lab2 (b) Prune by percentage

In [4]:
def prune_by_percentage(layer, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least
    significant weight parameters will be pruned.
    """
    # Convert the weight of "layer" to numpy array
    weight = layer.weight.data.cpu().numpy()

    # Compute the q-th percentile of the abs of the converted array
    threshold = np.percentile(np.abs(weight), q)
    # Generate a binary mask same shape as weight to decide which element to prune
    mask = np.where(np.abs(weight) < threshold, 0, 1)
    # Convert mask to torch tensor and put on GPU
    mask = torch.tensor(mask).to(device)
    # Multiply the weight by mask to perform pruning
    layer.weight.data = layer.weight.data * mask
    pass


In [5]:
pruning_percentages = [20, 40, 60, 70, 80]

for q in pruning_percentages:
    # Reload the full model checkpoint
    net.load_state_dict(torch.load("pretrained_model.pt"))
    net = net.to(device)

    # Apply pruning by percentage
    for name, layer in net.named_modules():
        if isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear):
            prune_by_percentage(layer, q)
            # Optional: Check the sparsity achieved in each layer
            np_weight = layer.weight.data.cpu().numpy()
            zeros = np.sum(np_weight == 0)
            total = np_weight.size
            print('Sparsity of {} at q={}: {:.2f}%'.format(name, q, (zeros / total) * 100))

    # Test the model
    test_accuracy = test(net)


Sparsity of head_conv.0.conv at q=20: 20.14%
Sparsity of body_op.0.conv1.0.conv at q=20: 20.01%
Sparsity of body_op.0.conv2.0.conv at q=20: 20.01%
Sparsity of body_op.1.conv1.0.conv at q=20: 20.01%
Sparsity of body_op.1.conv2.0.conv at q=20: 20.01%
Sparsity of body_op.2.conv1.0.conv at q=20: 20.01%
Sparsity of body_op.2.conv2.0.conv at q=20: 20.01%
Sparsity of body_op.3.conv1.0.conv at q=20: 20.01%
Sparsity of body_op.3.conv2.0.conv at q=20: 20.00%
Sparsity of body_op.3.id_mapping.0.conv at q=20: 20.12%
Sparsity of body_op.4.conv1.0.conv at q=20: 20.00%
Sparsity of body_op.4.conv2.0.conv at q=20: 20.00%
Sparsity of body_op.5.conv1.0.conv at q=20: 20.00%
Sparsity of body_op.5.conv2.0.conv at q=20: 20.00%
Sparsity of body_op.6.conv1.0.conv at q=20: 20.00%
Sparsity of body_op.6.conv2.0.conv at q=20: 20.00%
Sparsity of body_op.6.id_mapping.0.conv at q=20: 20.02%
Sparsity of body_op.7.conv1.0.conv at q=20: 20.00%
Sparsity of body_op.7.conv2.0.conv at q=20: 20.00%
Sparsity of body_op.8.conv1

### Lab2 (c) Finetune pruned model

In [6]:
def finetune_after_prune(net, trainloader, criterion, optimizer, prune=True):
    """
    Finetune the pruned model for a single epoch
    Make sure pruned weights are kept as zero
    """
    # Build a dictionary for the nonzero weights
    weight_mask = {}
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Your code here: generate a mask in GPU torch tensor to have 1 for nonzero element and 0 for zero element
            weight_mask[name] = (layer.weight.data !=0).int().to(device)

    global_steps = 0
    train_loss = 0
    correct = 0
    total = 0
    start = time.time()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        if prune:
            for name,layer in net.named_modules():
                if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                    # Your code here: Use weight_mask to make sure zero elements remains zero
                    layer.weight.data = layer.weight.data * weight_mask[name]

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        global_steps += 1

        if global_steps % 50 == 0:
            end = time.time()
            batch_size = 256
            num_examples_per_second = 50 * batch_size / (end - start)
            print("[Step=%d]\tLoss=%.4f\tacc=%.4f\t%.1f examples/second"
                 % (global_steps, train_loss / (batch_idx + 1), (correct / total), num_examples_per_second))
            start = time.time()



In [7]:
# Get pruned model
net.load_state_dict(torch.load("pretrained_model.pt"))
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        prune_by_percentage(layer, q=80.0)

# Training setup, do not change
batch_size=256
lr=0.002
reg=1e-4

print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=16)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.875, weight_decay=reg, nesterov=False)

==> Preparing data..


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [8]:
# Model finetuning
for epoch in range(20):
    print('\nEpoch: %d' % epoch)
    net.train()
    finetune_after_prune(net, trainloader, criterion, optimizer)
    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if val_acc > best_acc:
        best_acc = val_acc
        print("Saving...")
        torch.save(net.state_dict(), "net_after_finetune.pt")


Epoch: 0
[Step=50]	Loss=0.9803	acc=0.6842	1527.4 examples/second
[Step=100]	Loss=0.8045	acc=0.7366	2867.6 examples/second
[Step=150]	Loss=0.7190	acc=0.7623	2282.1 examples/second
Test Loss=0.5568, Test acc=0.8169
Saving...

Epoch: 1
[Step=50]	Loss=0.4786	acc=0.8396	1598.2 examples/second
[Step=100]	Loss=0.4596	acc=0.8451	2866.2 examples/second
[Step=150]	Loss=0.4511	acc=0.8480	2167.1 examples/second
Test Loss=0.4857, Test acc=0.8393
Saving...

Epoch: 2
[Step=50]	Loss=0.3929	acc=0.8648	1588.5 examples/second
[Step=100]	Loss=0.3916	acc=0.8648	2845.4 examples/second
[Step=150]	Loss=0.3908	acc=0.8647	2815.1 examples/second
Test Loss=0.4530, Test acc=0.8508
Saving...

Epoch: 3
[Step=50]	Loss=0.3729	acc=0.8695	1692.3 examples/second
[Step=100]	Loss=0.3663	acc=0.8732	2388.1 examples/second
[Step=150]	Loss=0.3633	acc=0.8746	2865.0 examples/second
Test Loss=0.4344, Test acc=0.8563
Saving...

Epoch: 4
[Step=50]	Loss=0.3430	acc=0.8823	1838.0 examples/second
[Step=100]	Loss=0.3464	acc=0.8810	2310

In [9]:
# Check sparsity of the finetuned model, make sure it's not changed
net.load_state_dict(torch.load("net_after_finetune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = layer.weight.data.cpu().numpy()
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        # Print sparsity
        print('Sparsity of '+name+': '+str(zeros/total))

test(net)

Sparsity of head_conv.0.conv: 0.7986111111111112
Sparsity of body_op.0.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.0.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.1.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.2.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.3.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.4.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv1.0.conv: 0.7999131944444444
Sparsity of body_op.5.conv2.0.conv: 0.7999131944444444
Sparsity of body_op.6.conv1.0.conv: 0.7999674479166666
Sparsity of body_op.6.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv1.0.conv: 0.7999945746527778
Sparsity of body_op.7.conv2.0.conv: 0.7999945746527778
Sparsity of body_op.8.conv1.0.conv: 0.7999945746527778
Sparsity of body

### Lab2 (d) Iterative pruning

In [10]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)

    net.train()
    if epoch<10:
        for name,layer in net.named_modules():
            if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
                # Increase model sparsity
                q = epoch * 8
                prune_by_percentage(layer, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)

    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0484	acc=0.9841	1816.0 examples/second
[Step=100]	Loss=0.0485	acc=0.9844	2305.3 examples/second
[Step=150]	Loss=0.0480	acc=0.9848	2818.3 examples/second
Test Loss=0.3245, Test acc=0.9148

Epoch: 1
[Step=50]	Loss=0.0451	acc=0.9869	1707.1 examples/second
[Step=100]	Loss=0.0467	acc=0.9855	2332.0 examples/second
[Step=150]	Loss=0.0472	acc=0.9850	2822.2 examples/second
Test Loss=0.3257, Test acc=0.9148

Epoch: 2
[Step=50]	Loss=0.0511	acc=0.9830	1827.7 examples/second
[Step=100]	Loss=0.0501	acc=0.9840	2276.3 examples/second
[Step=150]	Loss=0.0508	acc=0.9832	2627.3 examples/second
Test Loss=0.3278, Test acc=0.9135

Epoch: 3
[Step=50]	Loss=0.0535	acc=0.9829	1621.8 examples/second
[Step=100]	Loss=0.0525	acc=0.9830	2784.4 examples/second
[Step=150]	Loss=0.0532	acc=0.9832	2206.8 examples/second
Test Loss=0.3284, Test acc=0.9129

Epoch: 4
[Step=50]	Loss=0.0575	acc=0.9809	1587.4 examples/second
[Step=100]	Loss=0.0561	acc=0.9812	2827.8 examples/second
[Step=150]	Loss=0.057

In [11]:
# Check sparsity of the final model, make sure it's 80%
net.load_state_dict(torch.load("net_after_iterative_prune.pt"))

for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here: can copy from previous question
        np_weight = layer.weight.data.cpu().numpy()
        zeros = np.sum(np_weight == 0)
        total = np_weight.size
        print('Sparsity of ' + name + ': ' + str(zeros / total))


test(net)

Sparsity of head_conv.0.conv: 0.7199074074074074
Sparsity of body_op.0.conv1.0.conv: 0.7200520833333334
Sparsity of body_op.0.conv2.0.conv: 0.7200520833333334
Sparsity of body_op.1.conv1.0.conv: 0.7200520833333334
Sparsity of body_op.1.conv2.0.conv: 0.7200520833333334
Sparsity of body_op.2.conv1.0.conv: 0.7200520833333334
Sparsity of body_op.2.conv2.0.conv: 0.7200520833333334
Sparsity of body_op.3.conv1.0.conv: 0.7200520833333334
Sparsity of body_op.3.conv2.0.conv: 0.7199435763888888
Sparsity of body_op.4.conv1.0.conv: 0.7199435763888888
Sparsity of body_op.4.conv2.0.conv: 0.7199435763888888
Sparsity of body_op.5.conv1.0.conv: 0.7199435763888888
Sparsity of body_op.5.conv2.0.conv: 0.7199435763888888
Sparsity of body_op.6.conv1.0.conv: 0.7199978298611112
Sparsity of body_op.6.conv2.0.conv: 0.7199978298611112
Sparsity of body_op.7.conv1.0.conv: 0.7199978298611112
Sparsity of body_op.7.conv2.0.conv: 0.7199978298611112
Sparsity of body_op.8.conv1.0.conv: 0.7199978298611112
Sparsity of body

### Lab2 (e) Global iterative pruning

In [12]:
def global_prune_by_percentage(net, q=70.0):
    """
    Pruning the weight paramters by threshold.
    :param q: pruning percentile. 'q' percent of the least
    significant weight parameters will be pruned.
    """
    # A list to gather all the weights
    flattened_weights = []
    # Find global pruning threshold
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weight = layer.weight.data.cpu().numpy()
            # Flatten the weight and append to flattened_weights
            flattened_weights.append(weight.flatten())

    # Concate all weights into a np array
    flattened_weights = np.concatenate(flattened_weights)
    # Find global pruning threshold
    thres = np.percentile(np.abs(flattened_weights), q)

    # Apply pruning threshold to all layers
    for name,layer in net.named_modules():
        if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
            # Convert weight to numpy
            weight = layer.weight.data.cpu().numpy()

            # Generate a binary mask same shape as weight to decide which element to prune
            mask = np.where(np.abs(weight) < thres, 0, 1)

            # Convert mask to torch tensor and put on GPU
            mask = torch.tensor(mask).to(device)

            # Multiply the weight by mask to perform pruning
            layer.weight.data = layer.weight.data * mask

            pass


In [13]:
net.load_state_dict(torch.load("pretrained_model.pt"))
best_acc = 0.
for epoch in range(20):
    print('\nEpoch: %d' % epoch)

    net.train()
    if epoch<10:
        # Increase model sparsity
        q = epoch * 8
        global_prune_by_percentage(net, q=q)
    if epoch<9:
        finetune_after_prune(net, trainloader, criterion, optimizer,prune=False)
    else:
        finetune_after_prune(net, trainloader, criterion, optimizer)

    #Start the testing code.
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    num_val_steps = len(testloader)
    val_acc = correct / total
    print("Test Loss=%.4f, Test acc=%.4f" % (test_loss / (num_val_steps), val_acc))

    if epoch>=10:
        if val_acc > best_acc:
            best_acc = val_acc
            print("Saving...")
            torch.save(net.state_dict(), "net_after_global_iterative_prune.pt")


Epoch: 0
[Step=50]	Loss=0.0473	acc=0.9849	1499.1 examples/second
[Step=100]	Loss=0.0485	acc=0.9841	2799.9 examples/second
[Step=150]	Loss=0.0482	acc=0.9843	2297.5 examples/second
Test Loss=0.3247, Test acc=0.9150

Epoch: 1
[Step=50]	Loss=0.0470	acc=0.9854	1569.8 examples/second
[Step=100]	Loss=0.0473	acc=0.9846	2876.5 examples/second
[Step=150]	Loss=0.0473	acc=0.9846	2309.3 examples/second
Test Loss=0.3269, Test acc=0.9143

Epoch: 2
[Step=50]	Loss=0.0456	acc=0.9852	1570.6 examples/second
[Step=100]	Loss=0.0467	acc=0.9850	2836.0 examples/second
[Step=150]	Loss=0.0469	acc=0.9851	2885.9 examples/second
Test Loss=0.3268, Test acc=0.9141

Epoch: 3
[Step=50]	Loss=0.0507	acc=0.9841	1776.6 examples/second
[Step=100]	Loss=0.0520	acc=0.9827	2416.5 examples/second
[Step=150]	Loss=0.0509	acc=0.9835	2855.5 examples/second
Test Loss=0.3277, Test acc=0.9144

Epoch: 4
[Step=50]	Loss=0.0566	acc=0.9800	1727.1 examples/second
[Step=100]	Loss=0.0544	acc=0.9816	2269.1 examples/second
[Step=150]	Loss=0.055

In [14]:
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))

zeros_sum = 0
total_sum = 0
for name,layer in net.named_modules():
    if (isinstance(layer, nn.Conv2d) or isinstance(layer, nn.Linear)) and 'id_mapping' not in name:
        # Your code here:
        # Convert the weight of "layer" to numpy array
        np_weight = np.abs(layer.weight.data.cpu().numpy())
        # Count number of zeros
        zeros = np.sum(np_weight == 0)
        # Count number of parameters
        total = np_weight.size
        zeros_sum+=zeros
        total_sum+=total
        print('Sparsity of '+name+': '+str(zeros/total))
print('Total sparsity of: '+str(zeros_sum/total_sum))
test(net)

Sparsity of head_conv.0.conv: 0.25462962962962965
Sparsity of body_op.0.conv1.0.conv: 0.5703125
Sparsity of body_op.0.conv2.0.conv: 0.5494791666666666
Sparsity of body_op.1.conv1.0.conv: 0.5347222222222222
Sparsity of body_op.1.conv2.0.conv: 0.5716145833333334
Sparsity of body_op.2.conv1.0.conv: 0.5447048611111112
Sparsity of body_op.2.conv2.0.conv: 0.58203125
Sparsity of body_op.3.conv1.0.conv: 0.5447048611111112
Sparsity of body_op.3.conv2.0.conv: 0.6012369791666666
Sparsity of body_op.4.conv1.0.conv: 0.6385633680555556
Sparsity of body_op.4.conv2.0.conv: 0.6975911458333334
Sparsity of body_op.5.conv1.0.conv: 0.6321614583333334
Sparsity of body_op.5.conv2.0.conv: 0.7259114583333334
Sparsity of body_op.6.conv1.0.conv: 0.6382378472222222
Sparsity of body_op.6.conv2.0.conv: 0.6733940972222222
Sparsity of body_op.7.conv1.0.conv: 0.6844075520833334
Sparsity of body_op.7.conv2.0.conv: 0.7421332465277778
Sparsity of body_op.8.conv1.0.conv: 0.7685004340277778
Sparsity of body_op.8.conv2.0.co

### Lab 3 (b) and (c): Fixed-point quantization

In [15]:
# Define quantized model and load weight
Nbits = 2 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)


Test Loss=9.5441, Test accuracy=0.0899


In [16]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..

Epoch: 0
[Step=50]	Loss=1.3331	acc=0.6201	1654.0 examples/second
[Step=100]	Loss=1.0715	acc=0.6823	2234.6 examples/second
[Step=150]	Loss=0.9395	acc=0.7132	2620.7 examples/second
Test Loss=0.7185, Test acc=0.7820
Saving...

Epoch: 1
[Step=200]	Loss=0.6605	acc=0.7705	1338.2 examples/second
[Step=250]	Loss=0.5900	acc=0.8010	2554.6 examples/second
[Step=300]	Loss=0.5633	acc=0.8099	2119.9 examples/second
[Step=350]	Loss=0.5473	acc=0.8153	2741.2 examples/second
Test Loss=0.6380, Test acc=0.8034
Saving...

Epoch: 2
[Step=400]	Loss=0.5039	acc=0.8247	1272.0 examples/second
[Step=450]	Loss=0.4777	acc=0.8371	2669.8 examples/second
[Step=500]	Loss=0.4632	acc=0.8421	2229.7 examples/second
[Step=550]	Loss=0.4575	acc=0.8437	2553.9 examples/second
Test Loss=0.6437, Test acc=0.8000

Epoch: 3
[Step=600]	Loss=0.4679	acc=0.8382	1267.4 examples/second
[Step=650]	Loss=0.4293	acc=0.8498	2643.2 examples/second
[Step=700]	Loss=0.4203	acc=0.8529	2284.7 examples/second
[Step=750]	Loss=0.41

### Lab3 (d) Quantize pruned model

In [17]:
# Define quantized model and load weight
Nbits = 5 #Change this value to finish (d)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits)
net = net.to(device)
net.load_state_dict(torch.load("net_after_global_iterative_prune.pt"))
test(net)


Test Loss=0.3212, Test accuracy=0.8952


In [18]:
# Quantized model finetuning
finetune(net, epochs=20, batch_size=256, lr=0.002, reg=1e-4)

# Load the model with best accuracy
net.load_state_dict(torch.load("quantized_net_after_finetune.pt"))
test(net)

==> Preparing data..

Epoch: 0
[Step=50]	Loss=0.1846	acc=0.9331	1598.1 examples/second
[Step=100]	Loss=0.1785	acc=0.9355	2177.2 examples/second
[Step=150]	Loss=0.1723	acc=0.9383	2534.3 examples/second
Test Loss=0.3404, Test acc=0.8939
Saving...

Epoch: 1
[Step=200]	Loss=0.1450	acc=0.9561	1269.9 examples/second
[Step=250]	Loss=0.1448	acc=0.9463	2461.2 examples/second
[Step=300]	Loss=0.1438	acc=0.9474	2184.0 examples/second
[Step=350]	Loss=0.1424	acc=0.9492	2556.6 examples/second
Test Loss=0.3339, Test acc=0.8976
Saving...

Epoch: 2
[Step=400]	Loss=0.1474	acc=0.9487	1271.3 examples/second
[Step=450]	Loss=0.1396	acc=0.9506	2503.2 examples/second
[Step=500]	Loss=0.1389	acc=0.9513	2213.1 examples/second
[Step=550]	Loss=0.1420	acc=0.9498	2738.2 examples/second
Test Loss=0.3214, Test acc=0.8982
Saving...

Epoch: 3
[Step=600]	Loss=0.1333	acc=0.9535	1231.2 examples/second
[Step=650]	Loss=0.1309	acc=0.9538	2634.1 examples/second
[Step=700]	Loss=0.1324	acc=0.9537	2248.6 examples/second
[Step=750]

### Lab4 (a) Symmetric quantization
#### Implement symmetric quantization in FP_layers.py, and repeat the process in (b)

In [24]:
# check the performance of symmetric quantization with 6, 5, 4, 3, 2 bits
Nbits = 6 #Change this value to finish (b) and (c)

net = ResNetCIFAR(num_layers=20, Nbits=Nbits, symmetric=True)
net = net.to(device)
net.load_state_dict(torch.load("pretrained_model.pt"))
test(net)

Test Loss=0.3231, Test accuracy=0.9146
